<picture>
  <source media="(prefers-color-scheme: dark)" srcset="https://vespa.ai/assets/vespa-ai-logo-heather.svg">
  <source media="(prefers-color-scheme: light)" srcset="https://vespa.ai/assets/vespa-ai-logo-rock.svg">
  <img alt="#Vespa" width="200" src="https://vespa.ai/assets/vespa-ai-logo-rock.svg" style="margin-bottom: 25px;">
</picture>

# Feeding to Vespa Cloud

Our [previous notebook](https://pyvespa.readthedocs.io/en/latest/examples/feed_performance.html), we demonstrated one way of benchmarking feed performance to a local Vespa instance running in Docker.
In this notebook, we will llok at the same methods, but how feeding to [Vespa Cloud](https://cloud.vespa.ai) affects performance of the different methods.

The key difference between feeding to a local Vespa instance and a Vespa Cloud instance is the network latency.
Additionally, we will introduce embedding in Vespa at feed time, which is a realistic scenario for many use-cases.

We will look at these 3 different methods:

1. Using `feed_iterable()` - which uses threading to parallelize the feed operation. Best for CPU-bound operations.
2. Using `feed_async_iterable()` - which uses asyncio to parallelize the feed operation. Also uses `httpx` with HTTP/2-support. Performs best for IO-bound operations.
3. Using [Vespa CLI](https://docs.vespa.ai/en/vespa-cli).


<div class="alert alert-info">
    Refer to <a href="https://pyvespa.readthedocs.io/en/latest/troubleshooting.html">troubleshooting</a>
    for any problem when running this guide.
</div>


Install [Vespa CLI](https://docs.vespa.ai/en/vespa-cli.html).
The `vespacli` python package is just a thin wrapper, allowing for installation through pypi.

> Do NOT install if you already have the Vespa CLI installed.


[Install pyvespa](https://pyvespa.readthedocs.io/), and other dependencies.


In [1]:
!pip3 install vespacli pyvespa datasets plotly>=5.20

zsh:1: 5.20 not found


## Create an application package

The [application package](https://pyvespa.readthedocs.io/en/latest/reference-api.html#vespa.package.ApplicationPackage)
has all the Vespa configuration files.

For this demo, we will use a simple application package


In [1]:
from vespa.package import (
    ApplicationPackage,
    Field,
    Schema,
    Document,
    FieldSet,
    HNSW,
)
from vespa.package import AuthClient, Parameter


# Define the application name (can NOT contain `_` or `-`)

application = "queryprofiling"


schema = Schema(
    name="doc",
    document=Document(
        fields=[
            Field(name="id", type="string", indexing=["summary"]),
            Field(name="text", type="string", indexing=["index", "summary"]),
            Field(name="long_text", type="string", indexing=["summary"]),
            Field(
                name="embedding",
                type="tensor<float>(x[1024])",
                # Note that we are NOT embedding with a vespa model here, but that is also possible.
                indexing=["summary", "attribute", "index"],
                ann=HNSW(distance_metric="angular"),
            ),
        ]
    ),
    fieldsets=[FieldSet(name="default", fields=["text"])],
)


CLIENT_TOKEN_ID = "pyvespa_integration"  # os.environ.get("VESPA_CLIENT_TOKEN_ID")
# Same as token name from the Vespa Cloud Console
auth_clients = [
    AuthClient(
        id="mtls",  # Note that you still need to include the mtls client.
        permissions=["read", "write"],
        parameters=[Parameter("certificate", {"file": "security/clients.pem"})],
    ),
    AuthClient(
        id="token",
        permissions=["read"],
        parameters=[Parameter("token", {"id": CLIENT_TOKEN_ID})],
    ),
]

app_package = ApplicationPackage(
    name=application, schema=[schema], auth_clients=auth_clients
)

Note that the `ApplicationPackage` name cannot have `-` or `_`.


## Deploy the Vespa application

Deploy `package` on the local machine using Docker,
without leaving the notebook, by creating an instance of
[VespaDocker](https://pyvespa.readthedocs.io/en/latest/reference-api.html#vespa.deployment.VespaDocker). `VespaDocker` connects
to the local Docker daemon socket and starts the [Vespa docker image](https://hub.docker.com/r/vespaengine/vespa/).

If this step fails, please check
that the Docker daemon is running, and that the Docker daemon socket can be used by clients (Configurable under advanced settings in Docker Desktop).


Follow the instrauctions from the output above and add the control-plane key in the console at `https://console.vespa-cloud.com/tenant/TENANT_NAME/account/keys`
(replace TENANT_NAME with your tenant name).


In [2]:
tenant_name = "vespa-team"

In [3]:
from vespa.deployment import VespaCloud
from vespa.application import Vespa
import os


def read_secret():
    """Read the API key from the environment variable. This is
    only used for CI/CD purposes."""
    t = os.getenv("VESPA_TEAM_API_KEY")
    if t:
        return t.replace(r"\n", "\n")
    else:
        return t


vespa_cloud = VespaCloud(
    tenant=tenant_name,
    application=application,
    key_content=read_secret()
    if read_secret()
    else None,  # Can removed this for interactive control-plane login
    application_package=app_package,
)

Setting application...
Running: vespa config set application vespa-team.queryprofiling
Setting target cloud...
Running: vespa config set target cloud

Api-key found for control plane access. Using api-key.


`app` now holds a reference to a [VespaCloud](https://pyvespa.readthedocs.io/en/latest/reference-api.html#vespa.deployment.VespaCloud) instance.


In [111]:
app = vespa_cloud.deploy()

Deployment started in run 4 of dev-aws-us-east-1c for vespa-team.queryprofiling. This may take a few minutes the first time.
INFO    [06:06:51]  Deploying platform version 8.400.15 and application dev build 3 for dev-aws-us-east-1c of default ...
INFO    [06:06:51]  Using CA signed certificate version 1
INFO    [06:06:51]  Using 1 nodes in container cluster 'queryprofiling_container'
INFO    [06:07:01]  Session 305572 for tenant 'vespa-team' prepared and activated.
INFO    [06:07:01]  ######## Details for all nodes ########
INFO    [06:07:01]  h96882a.dev.aws-us-east-1c.vespa-external.aws.oath.cloud: expected to be UP
INFO    [06:07:01]  --- platform vespa/cloud-tenant-rhel8:8.400.15
INFO    [06:07:01]  --- container on port 4080 has config generation 305571, wanted is 305572
INFO    [06:07:01]  --- metricsproxy-container on port 19092 has config generation 305572, wanted is 305572
INFO    [06:07:01]  h96539b.dev.aws-us-east-1c.vespa-external.aws.oath.cloud: expected to be UP
INFO    [

Vespa(https://ed847e76.c07901b1.z.vespa-app.cloud/)

In [4]:
mtls_endpoint = vespa_cloud.get_mtls_endpoint()
token_endpoint = vespa_cloud.get_token_endpoint()

Found mtls endpoint for queryprofiling_container
URL: https://ed847e76.c07901b1.z.vespa-app.cloud/
Found token endpoint for queryprofiling_container
URL: https://b94950e8.c07901b1.z.vespa-app.cloud/


## Preparing the data

In this example we use [HF Datasets](https://huggingface.co/docs/datasets/index) library to stream the
["Cohere/wikipedia-2023-11-embed-multilingual-v3"](https://huggingface.co/datasets/Cohere/wikipedia-2023-11-embed-multilingual-v3) dataset and index in our newly deployed Vespa instance.

The dataset contains wikipedia-pages, and their corresponding embeddings.

> For this exploration we will use the `id` , `text` and `embedding`-fields

The following uses the [stream](https://huggingface.co/docs/datasets/stream) option of datasets to stream the data without
downloading all the contents locally.

The `map` functionality allows us to convert the
dataset fields into the expected feed format for `pyvespa` which expects a dict with the keys `id` and `fields`:

`{ "id": "vespa-document-id", "fields": {"vespa_field": "vespa-field-value"}}`


In [5]:
from datasets import load_dataset

/Users/thomas/.pyenv/versions/3.9.19/envs/pyvespa-dev/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Utility function to create dataset with different number of documents


In [6]:
def get_dataset(n_docs: int = 1000):
    dataset = load_dataset(
        "Cohere/wikipedia-2023-11-embed-multilingual-v3",
        "simple",
        split=f"train[:{n_docs}]",
    )
    dataset = dataset.map(
        lambda x: {
            "id": x["_id"] + "-iter",
            "fields": {
                "text": x["text"],
                "embedding": x["emb"],
                "long_text": x["text"] * 100,
            },
        }
    ).select_columns(["id", "fields"])
    return dataset

In [8]:
from vespa.io import VespaResponse


def callback(response: VespaResponse, id: str):
    if not response.is_successful():
        print(
            f"Failed to feed document {id} with status code {response.status_code}: Reason {response.get_json()}"
        )

In [4]:
import nest_asyncio

nest_asyncio.apply()

In [11]:
app.feed_async_iterable(
    get_dataset(),
    schema="doc",
    namespace="pyvespa-feed",
    operation_type="feed",
    callback=callback,
)

In [5]:
query_body = {
    "yql": "select * from doc where true;",
    "hits": 10,
    "query": "a",
    "presentation.timing": "true",
    "input.query(q)": "a" * 1000,  # change this to change size
}

In [17]:
def get_token_app():
    return Vespa(
        url=token_endpoint,
        vespa_cloud_secret_token=os.getenv("VESPA_CLOUD_SECRET_TOKEN"),
    )


# Get user home directory
home = os.path.expanduser("~")
# Vespa key/cert directory
app_dir = f"{home}/.vespa/{tenant_name}.{application}.default/"

cert_path = f"{app_dir}/data-plane-public-cert.pem"
key_path = f"{app_dir}/data-plane-private-key.pem"


def get_mtls_app():
    return Vespa(
        url=mtls_endpoint,
        key=key_path,
        cert=cert_path,
    )

In [18]:
import pyinstrument

app = get_mtls_app()

with pyinstrument.profile():
    with app.syncio(connections=1, compress=True) as session:
        response = session.query(
            body=query_body,
        )

Using Mutual TLS with key and cert to connect to Vespa endpoint https://ed847e76.c07901b1.z.vespa-app.cloud/


Compressing request body

pyinstrument ........................................
.
.  Block at /var/folders/vb/ch14y_kn4mqfz75bhc9_g5980000gn/T/ipykernel_71508/1715205813.py:5
.
.  0.941 <module>  ../../../../../../../var/folders/vb/ch14y_kn4mqfz75bhc9_g5980000gn/T/ipykernel_71508/1715205813.py:5
.  └─ 0.939 VespaSync.query  ../../../../vespa/application.py:1215
.     └─ 0.935 Session.post  requests/sessions.py:626
.           [10 frames hidden]  requests, urllib3, http, socket, ssl
.              0.934 Session.send  requests/sessions.py:673
.              └─ 0.541 CustomHTTPAdapter.send  ../../../../vespa/application.py:1032
.                 └─ 0.541 CustomHTTPAdapter.send  requests/adapters.py:434
.                       [22 frames hidden]  urllib3, ssl, <built-in>, http, socket
.                          0.264 _SSLSocket.do_handshake  <built-in>
.                 0.393 _SSLSocket.read  <built-in>
.  
.....................................................



<bound method VespaQueryResponse.get_json of <vespa.io.VespaQueryResponse object at 0x1125dbb50>>

In [19]:
import os

token = os.getenv("VESPA_CLOUD_SECRET_TOKEN")

In [8]:
import requests
import time

# Define the URL to be tested
url = token_endpoint + "/ApplicationStatus"
headers = {"Authorization ": f"Bearer {token}"}

# Capture the start time for DNS lookup
start_time = time.time()

with pyinstrument.profile():
    # Perform DNS lookup and connection
    response = requests.post(url, headers=headers, json=query_body)

# Capture various timing metrics
time_namelookup = response.elapsed.total_seconds()
time_connect = time.time() - start_time
time_appconnect = (
    time_connect  # In the context of plain HTTP, appconnect and connect are the same
)
time_pretransfer = time_connect
time_starttransfer = response.elapsed.total_seconds()
time_total = time.time() - start_time

# Print the timing metrics
print(f"Time_Namelookup: {time_namelookup:.6f}")
print(f"Time_Connect: {time_connect:.6f}")
print(f"Time_Appconnect: {time_appconnect:.6f}")
print(f"Time_Pretransfer: {time_pretransfer:.6f}")
print(f"Time_StartTransfer: {time_starttransfer:.6f}")
print(f"Time_Total: {time_total:.6f}")

Time_Namelookup: 0.531770
Time_Connect: 0.548399
Time_Appconnect: 0.548399
Time_Pretransfer: 0.548399
Time_StartTransfer: 0.531770
Time_Total: 0.548483



pyinstrument ........................................
.
.  Block at /var/folders/vb/ch14y_kn4mqfz75bhc9_g5980000gn/T/ipykernel_71508/2005960894.py:11
.
.  0.538 <module>  ../../../../../../../var/folders/vb/ch14y_kn4mqfz75bhc9_g5980000gn/T/ipykernel_71508/2005960894.py:11
.  └─ 0.538 post  requests/api.py:103
.        [25 frames hidden]  requests, urllib3, socket, ssl, <buil...
.           0.129 socket.connect  <built-in>
.           0.122 getaddrinfo  <built-in>
.           0.132 _SSLSocket.do_handshake  <built-in>
.           0.134 _SSLSocket.read  <built-in>
.  
.....................................................



In [9]:
from vespa.application import VespaSync

app = get_mtls_app()

with pyinstrument.profile():
    with VespaSync(app=app, pool_connections=1, compress=False) as session:
        response = session.query(
            body=query_body,
        )

NameError: name 'get_mtls_app' is not defined

In [97]:
import asyncio


async def query():
    async with app.asyncio(connections=1) as session:
        response = await session.query(
            body=query_body,
        )
    return response


with pyinstrument.profile():
    response = asyncio.run(query())


pyinstrument ........................................
.
.  Block at /var/folders/vb/ch14y_kn4mqfz75bhc9_g5980000gn/T/ipykernel_84789/2363228634.py:12
.
.  0.551 <module>  ../../../../../../../var/folders/vb/ch14y_kn4mqfz75bhc9_g5980000gn/T/ipykernel_84789/2363228634.py:12
.  └─ 0.551 run  nest_asyncio.py:25
.        [5 frames hidden]  nest_asyncio, selectors, asyncio
.           0.516 kqueue.control  <built-in>
.           0.027 Task.__step  asyncio/tasks.py:239
.           └─ 0.024 query  ../../../../../../../var/folders/vb/ch14y_kn4mqfz75bhc9_g5980000gn/T/ipykernel_84789/2363228634.py:4
.              ├─ 0.017 async_wrapped  tenacity/asyncio/__init__.py:181
.              │     [0 frames hidden]  
.              │        0.017 AsyncRetrying.__call__  tenacity/asyncio/__init__.py:104
.              │        └─ 0.016 VespaAsync.query  ../../../../vespa/application.py:1568
.              │           └─ 0.015 AsyncClient.post  httpx/_client.py:1871
.              │                 [9 fr

In [139]:
import time
from concurrent.futures import ThreadPoolExecutor
import numpy as np


# Function to perform a single synchronous request
def sync_request(app, query_body, compress):
    start_time = time.time()
    with app.syncio(connections=1, compress=compress) as session:
        response = session.query(body=query_body)
    end_time = time.time()
    return {
        "searchtime": response.get_json()["timing"]["searchtime"],
        "e2e-time": end_time - start_time,
    }


# Function to run multiple requests in parallel
def run_parallel_requests(n, app, query_body, compress):
    with ThreadPoolExecutor(max_workers=n) as executor:
        futures = [
            executor.submit(sync_request, app, query_body, compress) for _ in range(n)
        ]
        results = [future.result() for future in futures]
    return results


n = 30
app = get_token_app()  # get_token_app()
compress = False
# Execute 30 parallel requests
with pyinstrument.profile():
    results = run_parallel_requests(n, app, query_body, compress)

# Gather p50 and p99 search time and e2e-time
searchtime = [r["searchtime"] for r in results]
e2e_time = [r["e2e-time"] for r in results]

p50_searchtime = np.percentile(searchtime, 50)
p99_searchtime = np.percentile(searchtime, 99)
p50_e2e_time = np.percentile(e2e_time, 50)
p99_e2e_time = np.percentile(e2e_time, 99)

print(f"p50 search time: {p50_searchtime:.2f} ms")
print(f"p99 search time: {p99_searchtime:.2f} ms")
print(f"p50 e2e time: {p50_e2e_time:.2f} ms")
print(f"p99 e2e time: {p99_e2e_time:.2f} ms")

Using Vespa Cloud Secret Token to connect to Vespa endpoint https://b94950e8.c07901b1.z.vespa-app.cloud/
Using Vespa Cloud Secret Token to connect to Vespa endpoint https://b94950e8.c07901b1.z.vespa-app.cloud/
Using Vespa Cloud Secret Token to connect to Vespa endpoint https://b94950e8.c07901b1.z.vespa-app.cloud/
Using Vespa Cloud Secret Token to connect to Vespa endpoint https://b94950e8.c07901b1.z.vespa-app.cloud/
Using Vespa Cloud Secret Token to connect to Vespa endpoint https://b94950e8.c07901b1.z.vespa-app.cloud/
Using Vespa Cloud Secret Token to connect to Vespa endpoint https://b94950e8.c07901b1.z.vespa-app.cloud/
Using Vespa Cloud Secret Token to connect to Vespa endpoint https://b94950e8.c07901b1.z.vespa-app.cloud/
Using Vespa Cloud Secret Token to connect to Vespa endpoint https://b94950e8.c07901b1.z.vespa-app.cloud/
Using Vespa Cloud Secret Token to connect to Vespa endpoint https://b94950e8.c07901b1.z.vespa-app.cloud/
Using Vespa Cloud Secret Token to connect to Vespa endp


pyinstrument ........................................
.
.  Block at /var/folders/vb/ch14y_kn4mqfz75bhc9_g5980000gn/T/ipykernel_84789/2413084013.py:29
.
.  2.022 <module>  ../../../../../../../var/folders/vb/ch14y_kn4mqfz75bhc9_g5980000gn/T/ipykernel_84789/2413084013.py:29
.  └─ 2.021 run_parallel_requests  ../../../../../../../var/folders/vb/ch14y_kn4mqfz75bhc9_g5980000gn/T/ipykernel_84789/2413084013.py:16
.     └─ 2.003 <listcomp>  ../../../../../../../var/folders/vb/ch14y_kn4mqfz75bhc9_g5980000gn/T/ipykernel_84789/2413084013.py:22
.        └─ 2.003 Future.result  concurrent/futures/_base.py:418
.              [1 frames hidden]  threading
.                 2.003 lock.acquire  <built-in>
.  
.....................................................



In [140]:
import time
import pandas as pd
import numpy as np
import asyncio

del sync_request


# Function to perform a single synchronous request
def sync_request(app, query_body, compress):
    start_time = time.time()
    with app.syncio(connections=1, compress=compress) as session:
        response = session.query(body=query_body)
    end_time = time.time()
    return {
        "searchtime": response.get_json()["timing"]["searchtime"],
        "e2e-time": end_time - start_time,
    }


# Function to perform a single asynchronous request
async def async_request(app, query_body, compress):
    start_time = time.time()
    async with app.asyncio(connections=1) as session:
        response = await session.query(body=query_body)
    end_time = time.time()
    return {
        "searchtime": response.get_json()["timing"]["searchtime"],
        "e2e-time": end_time - start_time,
    }


# Function to calculate statistics (p50, p99) for single request
def calculate_statistics(times):
    searchtimes = [t["searchtime"] for t in times]
    e2e_times = [t["e2e-time"] for t in times if t["e2e-time"] is not None]

    return {
        "p50_searchtime": np.percentile(searchtimes, 50),
        "p99_searchtime": np.percentile(searchtimes, 99),
        "p50_e2e_time": np.percentile(e2e_times, 50) if e2e_times else None,
        "p99_e2e_time": np.percentile(e2e_times, 99) if e2e_times else None,
    }


# Main function to perform benchmarking
def benchmark(vespa_cloud, test_cases):
    results = []

    for label, data in test_cases:
        for app_type, app_func in [("token", get_token_app), ("mtls", get_mtls_app)]:
            app = app_func()
            # Without compression
            times = [sync_request(app, data, compress=False)]
            stats = calculate_statistics(times)
            results.append(
                {"Size": label, "Compression": "No", **stats, "Auth": app_type}
            )

            # Delete the app and reinitialize
            del app
            app = vespa_cloud.get_application()

            # With compression
            times = [sync_request(app, data, compress=True)]
            stats = calculate_statistics(times)
            results.append(
                {"Size": label, "Compression": "Yes", **stats, "Auth": app_type}
            )

            # Delete the app and reinitialize
            del app
            app = vespa_cloud.get_application()

            # With async request and compression
            times = [asyncio.run(async_request(app, data, compress=True))]
            stats = calculate_statistics(times)
            results.append(
                {"Size": label, "Compression": "Yes (Async)", **stats, "Auth": app_type}
            )

    # Convert results to DataFrame
    df = pd.DataFrame(results)
    return df


# Define the test cases as before
base_data = {
    "yql": "select * from doc where true;",
    "hits": 10,
    "query": "april",
    "timeout": "2s",
    "presentation.timing": "true",
}


def get_random_letter():
    return chr(np.random.randint(97, 123))


# Create sample data
data_512b = {
    **base_data,
    "input.query(q)": get_random_letter() * 512,
}

data_10kb = {
    **base_data,
    "input.query(q)": get_random_letter() * 10240,
}

data_1mb = {
    **base_data,
    "input.query(q)": get_random_letter() * 1024 * 1024,
}

# Test cases for string queries
test_cases = [
    ("512B", data_512b),
    ("10kB", data_10kb),
    ("1MB", data_1mb),
]

# Assuming 'vespa_cloud' is already initialized
# Perform the benchmarking
df = benchmark(vespa_cloud, test_cases)

Using Vespa Cloud Secret Token to connect to Vespa endpoint https://b94950e8.c07901b1.z.vespa-app.cloud/
Only region: aws-us-east-1c available in dev environment.
Found mtls endpoint for queryprofiling_container
URL: https://ed847e76.c07901b1.z.vespa-app.cloud/
Connecting to https://ed847e76.c07901b1.z.vespa-app.cloud/
Using Mutual TLS with key and cert to connect to Vespa endpoint https://ed847e76.c07901b1.z.vespa-app.cloud/


Compressing request body


Only region: aws-us-east-1c available in dev environment.
Found mtls endpoint for queryprofiling_container
URL: https://ed847e76.c07901b1.z.vespa-app.cloud/
Connecting to https://ed847e76.c07901b1.z.vespa-app.cloud/
Using Mutual TLS with key and cert to connect to Vespa endpoint https://ed847e76.c07901b1.z.vespa-app.cloud/
Using Mutual TLS with key and cert to connect to Vespa endpoint https://ed847e76.c07901b1.z.vespa-app.cloud/
Only region: aws-us-east-1c available in dev environment.
Found mtls endpoint for queryprofiling_container
URL: https://ed847e76.c07901b1.z.vespa-app.cloud/
Connecting to https://ed847e76.c07901b1.z.vespa-app.cloud/
Using Mutual TLS with key and cert to connect to Vespa endpoint https://ed847e76.c07901b1.z.vespa-app.cloud/


Compressing request body


Only region: aws-us-east-1c available in dev environment.
Found mtls endpoint for queryprofiling_container
URL: https://ed847e76.c07901b1.z.vespa-app.cloud/
Connecting to https://ed847e76.c07901b1.z.vespa-app.cloud/
Using Mutual TLS with key and cert to connect to Vespa endpoint https://ed847e76.c07901b1.z.vespa-app.cloud/
Using Vespa Cloud Secret Token to connect to Vespa endpoint https://b94950e8.c07901b1.z.vespa-app.cloud/
Only region: aws-us-east-1c available in dev environment.
Found mtls endpoint for queryprofiling_container
URL: https://ed847e76.c07901b1.z.vespa-app.cloud/
Connecting to https://ed847e76.c07901b1.z.vespa-app.cloud/
Using Mutual TLS with key and cert to connect to Vespa endpoint https://ed847e76.c07901b1.z.vespa-app.cloud/


...

Only region: aws-us-east-1c available in dev environment.
Found mtls endpoint for queryprofiling_container
URL: https://ed847e76.c07901b1.z.vespa-app.cloud/
Connecting to https://ed847e76.c07901b1.z.vespa-app.cloud/
Using Mutual TLS with key and cert to connect to Vespa endpoint https://ed847e76.c07901b1.z.vespa-app.cloud/
Using Vespa Cloud Secret Token to connect to Vespa endpoint https://b94950e8.c07901b1.z.vespa-app.cloud/
Only region: aws-us-east-1c available in dev environment.
Found mtls endpoint for queryprofiling_container
URL: https://ed847e76.c07901b1.z.vespa-app.cloud/
Connecting to https://ed847e76.c07901b1.z.vespa-app.cloud/
Using Mutual TLS with key and cert to connect to Vespa endpoint https://ed847e76.c07901b1.z.vespa-app.cloud/


Compressing request body


Only region: aws-us-east-1c available in dev environment.
Found mtls endpoint for queryprofiling_container
URL: https://ed847e76.c07901b1.z.vespa-app.cloud/
Connecting to https://ed847e76.c07901b1.z.vespa-app.cloud/
Using Mutual TLS with key and cert to connect to Vespa endpoint https://ed847e76.c07901b1.z.vespa-app.cloud/
Using Mutual TLS with key and cert to connect to Vespa endpoint https://ed847e76.c07901b1.z.vespa-app.cloud/
Only region: aws-us-east-1c available in dev environment.
Found mtls endpoint for queryprofiling_container
URL: https://ed847e76.c07901b1.z.vespa-app.cloud/
Connecting to https://ed847e76.c07901b1.z.vespa-app.cloud/
Using Mutual TLS with key and cert to connect to Vespa endpoint https://ed847e76.c07901b1.z.vespa-app.cloud/


Compressing request body


Only region: aws-us-east-1c available in dev environment.
Found mtls endpoint for queryprofiling_container
URL: https://ed847e76.c07901b1.z.vespa-app.cloud/
Connecting to https://ed847e76.c07901b1.z.vespa-app.cloud/
Using Mutual TLS with key and cert to connect to Vespa endpoint https://ed847e76.c07901b1.z.vespa-app.cloud/


In [141]:
df

,Size,Compression,p50_searchtime,p99_searchtime,p50_e2e_time,p99_e2e_time,Auth
0,512B,No,0.004,0.004,1.702048,1.702048,token
1,512B,Yes,0.004,0.004,2.685179,2.685179,token
2,512B,Yes (Async),0.003,0.003,3.684672,3.684672,token
3,512B,No,0.003,0.003,2.528865,2.528865,mtls
4,512B,Yes,0.003,0.003,2.589663,2.589663,mtls
5,512B,Yes (Async),0.003,0.003,2.628944,2.628944,mtls
6,10kB,No,0.013,0.013,1.614784,1.614784,token
7,10kB,Yes,0.003,0.003,2.616129,2.616129,token
8,10kB,Yes (Async),0.003,0.003,2.519848,2.519848,token
9,10kB,No,0.004,0.004,2.501156,2.501156,mtls


## Cleanup


In [26]:
vespa_cloud.delete()

Deactivated vespa-team.feedperformancecloud in dev.aws-us-east-1c
Deleted instance vespa-team.feedperformancecloud.default


## Next steps

Check out some of the other
[examples](https://pyvespa.readthedocs.io/en/latest/examples.html) in the documentation.
